In [1]:
#!pip install git+https://github.com/crazyfrogspb/RedditScore.git
#!apt-get install netbase

import string
import pandas as pd
import re
#from redditscore.tokenizer import CrazyTokenizer

#tokenizer = CrazyTokenizer(hashtags='split', urls='domain', twitter_handles='', stem='stem', pos_emojis=True, neg_emojis=True, neutral_emojis=True, normalize=2)

from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import train_test_split

tweets = pd.read_csv("/content/drive/My Drive/Colab Notebooks/data/raw_tweets_text.csv")
sentiments = pd.read_csv("/content/drive/My Drive/Colab Notebooks/data/t4sa_text_sentiment.csv",delimiter = "\t")

tweets.set_index(tweets.id, inplace=True)
sentiments.set_index(sentiments.TWID, inplace=True)
data=tweets.join(sentiments)
data.dropna(inplace=True)
data.drop(columns=['id', 'TWID'], inplace=True)
data["sentiment"] = data[['NEU', 'NEG', 'POS']].idxmax(axis=1)

data = data[['text', 'sentiment']].copy()
df_train, df_test = train_test_split(data, test_size=0.33, random_state=42)

df_train.text = df_train.text.str.lower()
df_test.text = df_test.text.str.lower()


def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

#df_train.text = df_train.text.apply(lambda x: remove_emoji(x))
#df_test.text = df_test.text.apply(lambda x: remove_emoji(x))
df_train.text = df_train.text.apply(lambda x:re.sub(r'http\S+', '', x))
df_test.text = df_test.text.apply(lambda x:re.sub(r'http\S+', '', x))

tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True)
df_train.text = df_train.text.apply(lambda x: tokenizer.tokenize(x))
df_test.text = df_test.text.apply(lambda x: tokenizer.tokenize(x))

df_train.text = df_train.text.apply(lambda x: ' '.join(x))
df_test.text = df_test.text.apply(lambda x: ' '.join(x))

df_train.text = df_train.text.map(lambda x : x.translate(str.maketrans('', '', string.punctuation)))
df_test.text = df_test.text.map(lambda x : x.translate(str.maketrans('', '', string.punctuation)))

df_train.text = df_train.text.str.replace("[0-9]", " ")
df_test.text = df_test.text.str.replace("[0-9]", " ")

df_train.text = df_train.text.str.strip(string.whitespace)
df_test.text = df_test.text.str.strip(string.whitespace)

df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [2]:
len(data)

1179957